In [2]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
## First, get the api key
with open('cfb_apikey.txt', 'r') as file:
    apikey = f"Bearer {file.read()}"

In [35]:
# specify base_url and endpoint url
url = "https://api.collegefootballdata.com/drives?seasonType=regular&year=2008&team=Utah&offense=Utah"

In [36]:
headers = {
    "Authorization": apikey,
    "Content-Type": "application/json"
}

In [37]:
r = requests.get(url, headers=headers)
print(r.ok)
# print(r.url)

True


In [40]:
my_dict = r.json()

In [48]:
df = pd.DataFrame(my_dict)
columns_wanted = ['defense', 'scoring', 'elapsed', 'plays', 'yards', 'drive_result']
df[columns_wanted]

,defense,scoring,elapsed,plays,yards,drive_result
0,Michigan,True,"{'minutes': 3, 'seconds': 42}",8,75,RUSHING TD
1,Michigan,True,"{'minutes': 6, 'seconds': 14}",14,66,FG GOOD
2,Michigan,True,"{'minutes': 2, 'seconds': 7}",7,47,FG GOOD
3,Michigan,True,"{'minutes': 3, 'seconds': 21}",8,58,FG GOOD
4,Michigan,False,"{'minutes': 1, 'seconds': 4}",3,-3,PUNT
...,...,...,...,...,...,...
151,BYU,False,"{'minutes': 2, 'seconds': 14}",3,6,PUNT
152,BYU,True,"{'seconds': 36, 'minutes': 0}",4,31,PASSING TD
153,BYU,True,"{'seconds': 4, 'minutes': 0}",1,4,PASSING TD
154,BYU,True,"{'minutes': 5, 'seconds': 24}",9,29,PASSING TD
